In [ ]:
# -*- coding: utf-8 -*-
"""
The fou.nc file is read and *variable* is extracted. Next, the mesh data is 
converted to a raster and saved to a .tiff.
Created on Fri December 1th 2023

@author: Hydrologic
"""

import os, sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
sys.path.append("..")

from model.dataloader.dataloader_flowmesh import load_map_data, mesh_to_tiff
from model.helper_functions.plotting import raster_plot_with_context

__CASE__ = '...'

# Initialize base directory
inputdir = '...' % __CASE__
outputdir = '...' % __CASE__

# Raster options
resolution = 20  # m
distance_tol = 36  # m
interpolation = r"nearest"

# Check if the folder doesn't already exist and create it
if not os.path.exists(os.path.join(outputdir, 'waterdepth_rasters')):
    os.makedirs(os.path.join(outputdir, 'waterdepth_rasters'))

# Get the value t0 from the structures.ini file
def get_t0_from_structures(filename):
    # Create an empty dictionary to store section data
    data, current_section = {}, None 

    # Check if structures.ini exists
    if os.path.exists(filename):
        # Open the ini file
        with open(filename, 'r') as file:
            # Read the file line by line
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace

                # Check if the line contains a section header (e.g., [SectionName])
                if line.startswith('[') and line.endswith(']'):
                    # Extract the section name
                    current_section = line[1:-1]

                    # Check if the section doesn't exist in the data dictionary
                    if current_section not in data:
                        # Create an empty list to store lines for this section
                        data[current_section] = []
                    
                # Check if the line belongs to a section (i.e., not a comment or empty line)
                elif current_section:
                    # Append the line to the list corresponding to the current section
                    data[current_section].append(line)
        # Check if there are one or two dambreaks
        if 'Structure' in data:                                
            for line in data['Structure'][-22:]:
                if line.startswith('T0'):
                    key, value = map(str.strip, line.split('='))
                    t0 = int(value)
        return t0
t0 = get_t0_from_structures(inputdir + '/structures.ini')

# Get time information from map file
map_ = nc.Dataset(inputdir + '/DFM_map.nc')

# Get the first timestamp after bres
t_start = np.where(map_['time'][:] > t0)[0][0] - 1

# load mesh coordinates and data from netCDF 
map_data = load_map_data(inputdir + '/DFM_map.nc', r"Mesh2d_waterdepth")

for timestep in range(t_start, map_data.shape[0]):
    # Print timestep
    print(timestep, map_data.shape[0])

    # convert to raster and save as tiff
    _, _, grid_data = mesh_to_tiff(
        map_data[timestep, :],
        inputdir + '/DFM_map.nc',
        os.path.join(outputdir, 'waterdepth_rasters') + '/dataset{:04d}.tiff'.format(timestep - t_start),
        resolution,
        distance_tol,
        interpolation=interpolation,
    )